In [1]:
from bcb import sgs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sidrapy as sidra
from functools import reduce
import requests
from io import BytesIO
import yfinance as yf
import os
import shutil

In [2]:

# Definindo o estilo do matplotlib para os gráficos
mpl.style.use('bmh')

def get_bcb_data():
    # Dicionário com as séries desejadas
    bcb_series = {
        'BCB: PIB Mensal': 4380,
        'BCB: Papel Moeda em Circulação': 1786,
        'BCB: Reservas Bancárias': 1787,
        'BCB: Papel Moeda em Poder do Público': 27789,
        'BCB: M1': 27791,
        'BCB: M2': 27810,
        'BCB: M3': 27813,
        'BCB: M4': 27815,
        'BCB: Taxa de Câmbio USD Média Mensal': 3697,
        'BCB: Índice de Commodities - Brasil - Agropecuária': 27575,
        'BCB: Índice de Commodities - Brasil - Metal': 27576,
        'BCB: Índice de Commodities - Brasil - Metal': 27577,
        'BCB: SELIC acumulado no mês': 4390,
        'FECOMERCIO: Índice de Confiança do Consumidor': 4393,
        'FECOMERCIO: ìndice de Condições Econômicas Atuais': 4394,
        'FGV: IGP-M': 189,
        'ANP: Produção de Derivados do Petróleo - Total': 1391
    }
    # Baixando os dados do Banco Central usando o `bcb` para todas as séries
    bcb_data = sgs.get(bcb_series, start='2003-01-01')
    bcb_data.index = bcb_data.index.to_period('M')
    return bcb_data


In [3]:
def sidra_series_adapt(df_sidra, column_name):
    # Remover as linhas que não contêm dados relevantes (supondo que sejam as duas primeiras linhas)
    df_sidra = df_sidra.iloc[2:].reset_index(drop=True)
    
    # Renomear as colunas abreviadas para nomes mais descritivos
    df_sidra.rename(columns={
        'D2C': 'Mês (Código)',
        'V': 'Valor'
    }, inplace=True)
    
    # Selecionar apenas as colunas necessárias
    df_sidra = df_sidra[['Mês (Código)', 'Valor']]
    
    # Converter 'Mês (Código)' em formato datetime
    df_sidra['Date'] = pd.to_datetime(df_sidra['Mês (Código)'], format='%Y%m')

    # Remover a coluna 'Mês (Código)', pois não é mais necessária
    df_sidra.drop(columns=['Mês (Código)'], inplace=True)
    
    # Definir 'Date' como índice
    df_sidra.set_index('Date', inplace=True)

    df_sidra.index = df_sidra.index.to_period('M')
    
    # Converter a coluna 'Valor' para numérico
    df_sidra['Valor'] = pd.to_numeric(df_sidra['Valor'], errors='coerce')

    # Renomear a coluna 'Valor' para o nome desejado
    df_sidra.rename(columns={'Valor': column_name}, inplace=True)

    if isinstance(df_sidra.columns, pd.MultiIndex):
        df_sidra.columns = [' '.join(col).strip() for col in df_sidra.columns.values]
    
    return df_sidra


def get_sidra_series(column_name: str, table_code:str, variable:str, classifications:dict) -> pd.DataFrame:
    
    """
    column_name: str
        Nome da coluna que será atribuída ao DataFrame final.
    table_code: str
        Código da tabela do SIDRA.
    variable: str
        Código da variável que será extraída.
    classifications: str
        Código da classificação que será extraída.
    """

    if classifications:
        df_sidra_brute = sidra.get_table(
            table_code=table_code, 
            territorial_level="1", 
            ibge_territorial_code="all", 
            period="all", 
            variable=variable, 
            classifications=classifications
        )
    else:
        df_sidra_brute = sidra.get_table(
            table_code=table_code, 
            territorial_level="1", 
            ibge_territorial_code="all", 
            period="all", 
            variable=variable
        )
    # Remover as linhas que não contêm dados relevantes (supondo que sejam as duas primeiras linhas)
    df_sidra = sidra_series_adapt(df_sidra_brute, column_name)

    return df_sidra

In [4]:
sidra_list = [
    {
        'column_name': 'IBGE: Indicador de Produção - Indústria de Transformação',
        'table_code': '8888',
        'variable': '12606',
        'classifications': {'544': '129316'}
    },
    {
        'column_name': 'IBGE: Indicador de Produção - Indústria Extrativa',
        'table_code': '8888',
        'variable': '12606',
        'classifications': {'544': '129315'}
    },
    {
        'column_name': 'IBGE: Indicador de Produção - Bens de Capital',
        'table_code': '8887',
        'variable': '12606',
        'classifications': {'543': '129278'}
    },
    {
        'column_name': 'IBGE: Indicador de Produção - Bens Intermediários',
        'table_code': '8887',
        'variable': '12606',
        'classifications': {'543': '129283'}
    },
    {
        'column_name': 'IBGE: Indicador de Produção - Bens de Consumo Duráveis',
        'table_code': '8887',
        'variable': '12606',
        'classifications': {'543': '129301'}
    },
    {
        'column_name': 'IBGE: Indicador de Produção - Bens de Consumo Semi-Duráveis',
        'table_code': '8887',
        'variable': '12606',
        'classifications': {'543': '129306'}
    },
    {
        'column_name': 'IBGE: Índice de Receita Nominal de Vendas no Comércio Varejista',
        'table_code': '8880',
        'variable': '7169',
        'classifications': {'11046': '56733'}
    },
    {
        'column_name': 'IBGE: Índice Nacional de Preços ao Consumidor Amplo (IPCA)',
        'table_code': '1737',
        'variable': '2266',
        'classifications': None
    },
    {
        'column_name': 'IBGE: Índice de Receita Nominal de Vendas de Material de Construção',
        'table_code': '8757',
        'variable': '7169',
        'classifications': {'11046': '56731'}
    }

]  

In [5]:
def get_sidra_data(sidra_list: list) -> pd.DataFrame:
    
    """
    sidra_list: list
        Lista de dicionários com as informações necessárias para baixar os dados do SIDRA.
    """
    
    # Inicializar um DataFrame vazio
    df_list = []
    
    for item in sidra_list:
        column_name = item['column_name']
        table_code = item['table_code']
        variable = item['variable']
        classifications = item['classifications']
        
        df_item = get_sidra_series(column_name, table_code, variable, classifications)
        
        df_list.append(df_item)

        df_merged = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True, how='outer'), df_list)

    # Interessado somente nos dados a partir do ano de 2003
    df_merged = df_merged[df_merged.index >= '2003-01-01']
    
    return df_merged

In [6]:
def get_anfavea_production() -> pd.DataFrame:
    """
    Faz o download do arquivo de séries temporais de produção de autoveículos da ANFAVEA,
    processa os dados e retorna um DataFrame com as categorias de produção selecionadas.
    
    Retorno:
        pd.DataFrame: DataFrame contendo as séries de produção para Automóveis,
                      Comerciais Leves, Caminhões e Ônibus.
    """
    # URL do arquivo Excel
    url = "https://anfavea.com.br/docs/SeriesTemporais_Autoveiculos.xlsm"
    
    # Cabeçalhos para simular um navegador
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
    }
    
    # Fazer o download do arquivo
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Levanta erro se o download falhar
    
    # Abrir o arquivo Excel
    file_content = BytesIO(response.content)
    
    # Combinar cabeçalhos (linha 4 e linha 5) para criar nomes descritivos
    headers_df = pd.read_excel(file_content, sheet_name='Séries_Temporais_Autoveículos', nrows=5, header=None)
    combined_headers = headers_df.iloc[3].ffill().astype(str) + ' - ' + headers_df.iloc[4].fillna('')
    
    # Ler os dados abaixo do cabeçalho combinado
    df_with_headers = pd.read_excel(
        file_content,
        sheet_name='Séries_Temporais_Autoveículos',
        header=5,  # Dados começam após as linhas de cabeçalho
        names=combined_headers.fillna(''),  # Garantir que todos os nomes de coluna sejam strings
    )
    
    # Identificar dinamicamente a coluna de datas
    date_column = df_with_headers.columns[0]  # Geralmente é a primeira coluna
    
    # Selecionar as colunas relacionadas à produção
    columns_of_interest = [col for col in df_with_headers.columns if 'Produção' in col]
    
    # Filtrar as colunas desejadas, incluindo a de data
    df_producao = df_with_headers.loc[:, [date_column] + columns_of_interest].copy()
    
    # Renomear a coluna de data para 'Data'
    df_producao.rename(columns={date_column: 'Data'}, inplace=True)
    
    # Ajustar os nomes das colunas para começar com "Anfavea:" e formatar adequadamente
    column_mapping = {col: f"Anfavea: Produção de {col.split(' - ')[0].capitalize()}" for col in columns_of_interest}
    df_producao.rename(columns=column_mapping, inplace=True)
    
    # Converter a coluna 'Data' para datetime e organizar como índice mensal
    df_producao['Data'] = pd.to_datetime(df_producao['Data'], errors='coerce')
    df_producao = df_producao.dropna(subset=['Data'])  # Remover linhas com datas inválidas
    df_producao.set_index(df_producao['Data'].dt.to_period('M'), inplace=True)
    df_producao.drop(columns=['Data'], inplace=True)
    
    # Converter colunas de produção para valores numéricos
    df_producao = df_producao.apply(pd.to_numeric, errors='coerce')

    df_producao = df_producao[df_producao.index >= '2003-01-01']
    
    return df_producao



In [7]:
def download_file(url: str, output_path: str):
    """
    Baixa um arquivo a partir da URL fornecida.
    """
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(output_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)
    print(f"Arquivo baixado: {output_path}")

def epe_mwh_consumption(file_path: str, sheet_name: str, row_identifier: str) -> pd.DataFrame:
    """
    Extrai dados mensais de consumo para uma linha específica (e.g., "TOTAL RESIDENCIAL") para todos os anos disponíveis.
    """
    # Carregar a aba
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None, engine="xlrd")
    
    # Encontrar as células com todas as ocorrências do identificador de linha
    row_indices = df[df[0] == row_identifier].index
    # Adicionar também aqueles com "TOTAL BRASIL"
    row_indices = row_indices.union(df[df[0] == "TOTAL BRASIL"].index)

    # Checa se o elemento da segunda coluna dos índices é "JAN", se for, acrescenta 1 ao índice (célula foi mesclada)
    row_indices = [row_index + 1 if df.iloc[row_index, 1] == "JAN" else row_index for row_index in row_indices]
    # Para cada uma dessas ocorrências, extrair o ano correspondente, o ano está na segunda coluna, duas linhas acima, e é um número de quatro dígitos
    # ou um número de quatro dígitos seguido de asterisco, associar portanto, a linha de ocorrencia do identificador de linha com o ano correspondente
    row_years = {row_index: df.iloc[row_index - 2, 1] for row_index in row_indices}
    # Para as ocorrências em que houver o asterisco, remover o asterisco, e transformar o valor em inteiro
    row_years = {row_index: int(str(year).replace("*", "")) for row_index, year in row_years.items()}
    # Os valores que estivem na linha de ocorrencia do identificador de linha, são os valores de consumo mensal e deve ser lido até o 12º mês
    row_values = {row_index: df.iloc[row_index, 1:13].values for row_index in row_indices}
    # Criar dataframe em que o índice é um period M (ano-mês) e a coluna é o valor de consumo mensal, na ordem 01, 02, ..., 12
    # Se o valor de consumo mensal for zero, a linha não é incluída
    data = {
        pd.Period(year=year, month=month, freq="M"): value
        for row_index, year in row_years.items()
        for month, value in enumerate(row_values[row_index], start=1)
        if value != 0
    }
    # O padrão do nome da coluna é EPE: Consumo de Energia Elétrica (Residencial, Industrial, Comercial) em MWh
    col_name = f"EPE: Consumo de Energia Elétrica ({sheet_name.capitalize()}) em MWh" 

    # Converter para dataframe
    df = pd.DataFrame(data, index=["Consumo"]).T
    df.columns = [col_name]
    # Ordena o índice do mais antigo para o mais novo
    df.sort_index(inplace=True)
    return df

def get_epe_data():
    # URL e caminho para download e extração dos dados
    url = "https://www.epe.gov.br/sites-pt/publicacoes-dados-abertos/publicacoes/Documents/CONSUMO%20MENSAL%20DE%20ENERGIA%20EL%C3%89TRICA%20POR%20CLASSE.xls"
    xls_path = "CONSUMO_MENSAL_ENERGIA.xls"

    # Baixar o arquivo
    download_file(url, xls_path)

    # Extrair dados para cada setor (Residencial, Industrial, Comercial)
    sectors = {
        "Residencial": {"sheet_name": "RESIDENCIAL", "row_identifier": "TOTAL RESIDENCIAL"},
        "Industrial": {"sheet_name": "INDUSTRIAL", "row_identifier": "TOTAL INDUSTRIAL (CATIVO + LIVRE)"},
        "Comercial": {"sheet_name": "COMERCIAL", "row_identifier": "TOTAL COMERCIAL (CATIVO + LIVRE)"},
    }

    # Extrai  para os setores e faz um merge em um dataframe único
    df_list = []
    for sector, params in sectors.items():
        df_sector = epe_mwh_consumption(xls_path, params["sheet_name"], params["row_identifier"])
        df_list.append(df_sector)

    df_merged = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True, how='outer'), df_list)
    # Apaga o arquivo
    os.remove(xls_path)
    return df_merged
    

In [8]:

def get_market_data():
    # Lista dos tickers selecionados
    tickers = {
        "^BVSP": "Mkt: Ibovespa",           # Ibovespa
        "CL=F": "Mkt: Petróleo (WTI)",      # Petróleo WTI
        "ZS=F": "Mkt: Soja",                # Soja
        "KC=F": "Mkt: Café",                # Café
        "ZC=F": "Mkt: Milho",               # Milho
        "SB=F": "Mkt: Açúcar",              # Açúcar
        "GC=F": "Mkt: Ouro",                # Ouro
        "^SPX": "Mkt: S&P 500",             # S&P 500
        "EEM": "Mkt: Mercados Emergentes",  # ETF de Mercados Emergentes
        "^DJT": "Mkt: Transportes EUA",     # Índice de Transportes Dow Jones
     }

    # Inicializa um DataFrame vazio para consolidar os dados
    all_data = pd.DataFrame()

    # Baixa os dados de cada ticker e adiciona ao DataFrame consolidado
    for ticker, label in tickers.items():
        print(f"Baixando dados para {label} {ticker}...")
        data = yf.download(ticker, start="2000-01-01", end="2025-12-31", interval="1mo")
        
        # Apenas a coluna de fechamento ajustado, renomeada para o indicador
        data = data[['Adj Close']].rename(columns={'Adj Close': label})
        
        # Ajusta o índice para períodos mensais
        data.index = data.index.to_period('M')
        
        # Combina com o DataFrame principal
        if all_data.empty:
            all_data = data
        else:
            all_data = all_data.join(data, how="outer")

    # Visualiza as primeiras linhas do DataFrame consolidado
    all_data = all_data[all_data.index >= '2003-01-01']

    # Faz Impute dos valores faltantes, interpolando linearmente
    all_data = all_data.interpolate(method='linear')

    print(all_data.head())
    return all_data


In [9]:
def download_and_process_rpk_data():
    base_url = "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/dados-estatisticos/arquivos/resumo_anual_{}.csv"
    start_year = 2003
    end_year = 2024  # Atualize conforme necessário
    combined_data = []

    for year in range(start_year, end_year + 1):
        url = base_url.format(year)
        local_file = f"resumo_anual_{year}.csv"
        
        # Download do arquivo
        response = requests.get(url)
        if response.status_code == 200:
            with open(local_file, "wb") as f:
                f.write(response.content)
            print(f"Baixado: {local_file}")

            # Carregar os dados
            try:
                data = pd.read_csv(local_file, encoding='latin1', sep=';', low_memory=False)
            except Exception as e:
                print(f"Erro ao processar o arquivo {local_file}: {e}")
                continue

            # Filtrar voos domésticos
            domestic_data = data.loc[
                (data['AEROPORTO DE ORIGEM (PAÍS)'] == 'BRASIL') &
                (data['AEROPORTO DE DESTINO (PAÍS)'] == 'BRASIL')
            ].copy()

            # Converter RPK para numérico
            domestic_data['RPK'] = pd.to_numeric(domestic_data['CARGA PAGA KM'], errors='coerce')

            # Agrupar por ano e mês e somar o RPK
            monthly_rpk = domestic_data.groupby(['ANO', 'MÊS'])['RPK'].sum().reset_index()
            combined_data.append(monthly_rpk)

            # Remover o arquivo baixado
            os.remove(local_file)
            print(f"Arquivo removido: {local_file}")
        else:
            print(f"Erro ao baixar o arquivo para o ano {year}: {response.status_code}")

    # Combinar todos os anos em um único DataFrame
    if combined_data:
        final_data = pd.concat(combined_data, ignore_index=True)
        # Criar uma coluna de período
        final_data['PERIOD'] = pd.to_datetime(final_data['ANO'].astype(str) + '-' + final_data['MÊS'].astype(str) + '-01')
        final_data['PERIOD'] = final_data['PERIOD'].dt.to_period('M')
        final_data = final_data[['PERIOD', 'RPK']].set_index('PERIOD')
        # Ordenar os dados
        final_data = final_data.sort_index()
        # O nome da coluna deve ser ANAC: RPK (Passageiro x Quilômetro)
        final_data.columns = ['ANAC: RPK (Passageiro x Quilômetro)']
        return final_data
    else:
        print("Nenhum dado foi processado.")
        return None



In [10]:
# Limites geográficos das regiões
regions = {
    "Norte": {"lat_min": 0, "lat_max": 5, "lon_min": -73, "lon_max": -45},
    "Nordeste": {"lat_min": -17, "lat_max": 0, "lon_min": -47, "lon_max": -35},
    "Centro-Oeste": {"lat_min": -24, "lat_max": -7, "lon_min": -61, "lon_max": -45},
    "Sudeste": {"lat_min": -24, "lat_max": -17, "lon_min": -48, "lon_max": -40},
    "Sul": {"lat_min": -34, "lat_max": -24, "lon_min": -57, "lon_max": -48},
}

def aggregate_spatially(df):
    """
    Agrega espacialmente os dados de todos os pontos da região.
    Calcula as médias diárias de cada variável.
    """

    # Média diária para todos os pontos
    daily_spatial_avg = df.groupby("date").mean().reset_index()
    
    # Remover as colunas de latitude e longitude, pois não são mais necessárias
    daily_spatial_avg = daily_spatial_avg.drop(columns=["latitude", "longitude"], errors="ignore")
    return daily_spatial_avg


def aggregate_monthly_data(df):
    """
    Agrega os dados diários em valores mensais:
    - Média para T2M
    - Mínimo para T2M_MIN
    - Máximo para T2M_MAX
    - Soma para PRECTOTCORR
    """
    # Converte para um índice de data para facilitar a manipulação
    df["month"] = df["date"].dt.to_period("M")

    # Agregação por mês
    monthly_data = df.groupby("month").agg({
        "T2M": "mean",              # Média da temperatura média
        "T2M_MIN": "min",           # Mínima da temperatura mínima
        "T2M_MAX": "max",           # Máxima da temperatura máxima
        "PRECTOTCORR": "sum",       # Soma da precipitação
    }).reset_index()

    # o month passará a ser o índice
    monthly_data.set_index("month", inplace=True)
    return monthly_data


def get_nasa_power_data(latitude, longitude, start_date, end_date):
    """
    Coleta dados históricos da NASA POWER API para um ponto específico.
    """
    url = f"https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        "parameters": "T2M,T2M_MIN,T2M_MAX,PRECTOTCORR",  # Temperatura e precipitação
        "community": "AG",
        "latitude": latitude,
        "longitude": longitude,
        "start": start_date,
        "end": end_date,
        "format": "JSON"
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()["properties"]["parameter"]
        df = pd.DataFrame(data)
        # Criar a coluna de data a partir do índice
        df["date"] = df.index
        # O índice é da forma '20030101', converter para date 
        df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
        return df
    else:
        print(f"Erro na API para ({latitude}, {longitude}):", response.status_code)
        return None

def process_region_with_aggregation(region_name, limits, start_date, end_date, step=4):
    """
    Processa os dados diários de uma região e os agrega em médias mensais.
    """
    lat_min, lat_max = limits["lat_min"], limits["lat_max"]
    lon_min, lon_max = limits["lon_min"], limits["lon_max"]

    latitudes = np.arange(lat_min, lat_max, step)
    longitudes = np.arange(lon_min, lon_max, step)

    data_frames = []

    # Coletar dados diários para cada ponto na grade
    for lat in latitudes:
        for lon in longitudes:
            df = get_nasa_power_data(lat, lon, start_date, end_date)
            if df is not None:
                df["latitude"] = lat
                df["longitude"] = lon
                data_frames.append(df)

    # Combinar dados de todos os pontos
    combined_df = pd.concat(data_frames)

    # Antes da consolidação, elimina as linhas em que pelo menos um dos valores é menor que -100 (valor inválido)
    # Imprime a data e a localização do ponto com valores inválidos
    invalid_data = combined_df[(combined_df["T2M"] < -100) | (combined_df["T2M_MIN"] < -100) | (combined_df["T2M_MAX"] < -100) | (combined_df["PRECTOTCORR"] < -100)]
    if not invalid_data.empty:
        print(f"Valores inválidos encontrados para {region_name}:")
        print(invalid_data[["date", "latitude", "longitude"]])

    combined_df = combined_df[(combined_df["T2M"] >= -100) & (combined_df["T2M_MIN"] >= -100) & (combined_df["T2M_MAX"] >= -100) & (combined_df["PRECTOTCORR"] >= -100)]

    spatially_aggregated = aggregate_spatially(combined_df)

    # Agregar os dados em médias mensais
    monthly_data = aggregate_monthly_data(spatially_aggregated)

    print(f"Dados agregados mensalmente para {region_name}")
    return monthly_data


def consolidate_regions(region_data):
    """
    Cria um DataFrame consolidado com colunas formatadas para cada variável e região.
    """
    consolidated_data = pd.DataFrame()

    for region_name, data in region_data.items():
        # Renomear colunas para incluir o nome da região
        data = data.rename(columns={
            "T2M_MIN": f"NASA Power: Temp. Min.(C)/{region_name}",
            "T2M_MAX": f"NASA Power: Temp. Máx.(C)/{region_name}",
            "PRECTOTCORR": f"NASA Power: Precipitação(mm)/{region_name}"
        })

        # Selecionar apenas as colunas necessárias
        data = data[[f"NASA Power: Temp. Min.(C)/{region_name}", f"NASA Power: Temp. Máx.(C)/{region_name}", f"NASA Power: Precipitação(mm)/{region_name}"]]

        # Concatenar com o DataFrame consolidado
        if consolidated_data.empty:
            consolidated_data = data
        else:
            consolidated_data = pd.concat([consolidated_data, data], axis=1)

    return consolidated_data


def get_climate_data():
    start_date = "20030101"
    end_date = "20241031"
    region_data = {}

    for region_name, limits in regions.items():
        print(f"Processando dados para a região: {region_name}")
        region_data[region_name] = process_region_with_aggregation(region_name, limits, start_date, end_date)

    consolidated_df = consolidate_regions(region_data)
    return consolidated_df


In [11]:
def get_comex_data():
    # URLs dos arquivos
    urls = {
        "exportacao": "https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/EXP_COMPLETA.zip",
        "importacao": "https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/IMP_COMPLETA.zip"
    }

    # Função para baixar, descompactar e processar os dados
    def process_comex_data(url, operation_type):
        print(f"Baixando dados de {operation_type}...")
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            # Salvar o arquivo ZIP localmente
            zip_path = f"temp_{operation_type}.zip"
            with open(zip_path, "wb") as file:
                file.write(response.content)

            # Extrair o conteúdo do arquivo ZIP
            extract_dir = f"extracted_{operation_type}"
            os.makedirs(extract_dir, exist_ok=True)
            shutil.unpack_archive(zip_path, extract_dir, "zip")

            # Localizar o arquivo CSV extraído
            csv_file = [f for f in os.listdir(extract_dir) if f.endswith(".csv")][0]
            csv_path = os.path.join(extract_dir, csv_file)

            # Ler o CSV e processar os dados
            print(f"Lendo o arquivo {csv_file} para {operation_type}...")
            df = pd.read_csv(csv_path, sep=";", usecols=["CO_ANO", "CO_MES", "VL_FOB"])

            # Criar a coluna de período e agregar os valores
            df["month"] = pd.to_datetime(df["CO_ANO"].astype(str) + df["CO_MES"].astype(str), format="%Y%m")

            df = df.groupby("month")["VL_FOB"].sum().sort_index().to_frame()

            # Renomear a coluna para identificar o tipo de operação
            df.columns = [f"VL_FOB_{operation_type}"]

            # Limpar arquivos temporários
            shutil.rmtree(extract_dir)
            os.remove(zip_path)

            return df
        else:
            print(f"Erro ao baixar {operation_type}: {response.status_code}")
            return pd.DataFrame()

    # Processar exportações e importações
    export_data = process_comex_data(urls["exportacao"], "exportacao")
    import_data = process_comex_data(urls["importacao"], "importacao")

    # Combinar os dados
    combined_data = pd.concat([export_data, import_data], axis=1).fillna(0)

    # O month já é o índice dos DataFrames export_data e import_data
    combined_data.index.name = "month"

    # Converter para period M
    combined_data.index = combined_data.index.to_period("M")
            
    # Ordenar pelo índice e salvar o arquivo consolidado
    combined_data = combined_data.sort_index()

    # Filtra para partir de 2003
    combined_data = combined_data[combined_data.index >= '2003-01-01']
    # A coluna de importação deve se chamar MDIC: Valor de Importação FOB e de exportação MDIC: Valor de Exportação FOB
    combined_data.columns = ["MDIC: Valor de Exportação (FOB)", "MDIC: Valor de Importação (FOB)"]
    return combined_data

In [12]:
def get_unemployment_data():
    # A URL do PME é da forma https://apisidra.ibge.gov.br/values/t/2179/n110/all/v/1035/p/last%201/d/v1035%201
    # Baixar com sidra.get_table
    df_pme_brute = sidra.get_table(
        table_code="2179",
        territorial_level="110",
        ibge_territorial_code="all",
        variable="1035",
        period="all",
        header=0
    )

    df_pme = sidra_series_adapt(df_pme_brute, "IBGE: Taxa de Desocupação")

    df_pnad = sidra.get_table(
        table_code="6381",
        territorial_level="1",
        ibge_territorial_code="all",
        variable="4099",
        period="all",
        header=0
    )

    df_pnad = sidra_series_adapt(df_pnad, "IBGE: Taxa de Desocupação")
    
    # Combinar, verticalmente, os dados do PME e PNAD, se a data for a mesma, preferir PNAD
    # Ou seja, um dataframe com data e taxa de desocupação é o resultado
    combined_unemployment_rate = pd.concat([df_pme, df_pnad]).sort_index()
    combined_unemployment_rate = combined_unemployment_rate[~combined_unemployment_rate.index.duplicated(keep='last')]

    # Filtrar para partir de 2003
    combined_unemployment_rate = combined_unemployment_rate[combined_unemployment_rate.index >= '2003-01-01']
    return combined_unemployment_rate


In [13]:
import concurrent.futures

def get_all_data():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submeter todas as funções get_data para execução paralela
        future_to_data = {
            executor.submit(get_bcb_data): 'bcb_data',
            executor.submit(get_anfavea_production): 'anfavea_data',
            executor.submit(get_sidra_data, sidra_list): 'sidra_data',
            executor.submit(get_epe_data): 'epe_data',
            executor.submit(get_market_data): 'market_data',
            executor.submit(download_and_process_rpk_data): 'rpk_data',
            executor.submit(get_climate_data): 'climate_data',
            executor.submit(get_comex_data): 'comex_data',
            executor.submit(get_unemployment_data): 'unemployment_data'
        }

        # Inicializar um dicionário para armazenar os resultados
        data_frames = {}

        # Coletar os resultados conforme as tarefas são concluídas
        for future in concurrent.futures.as_completed(future_to_data):
            data_name = future_to_data[future]
            try:
                data_frames[data_name] = future.result()
            except Exception as exc:
                print(f'{data_name} gerou uma exceção: {exc}')

    return data_frames

# Executar a função para obter todos os dados
data_frames = get_all_data()

# Fazer o join entre os DataFrames
all_data = pd.concat([
    data_frames['bcb_data'],
    data_frames['anfavea_data'],
    data_frames['sidra_data'],
    data_frames['epe_data'],
    data_frames['market_data'],
    data_frames['rpk_data'],
    data_frames['climate_data'],
    data_frames['comex_data'],
    data_frames['unemployment_data']
], axis=1)

Processando dados para a região: Norte
Baixando dados de exportacao...
Baixando dados para Mkt: Ibovespa ^BVSP...
Arquivo baixado: CONSUMO_MENSAL_ENERGIA.xls


[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_27638/3381059830.py:28: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data.index = data.index.to_period('M')


Baixando dados para Mkt: Petróleo (WTI) CL=F...


[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_27638/3381059830.py:28: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data.index = data.index.to_period('M')


Baixando dados para Mkt: Soja ZS=F...


...

Baixando dados para Mkt: Mercados Emergentes EEM...


[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_27638/3381059830.py:28: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data.index = data.index.to_period('M')


Baixando dados para Mkt: Transportes EUA ^DJT...


[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_27638/3381059830.py:28: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data.index = data.index.to_period('M')


Price   Mkt: Ibovespa Mkt: Petróleo (WTI) Mkt: Soja  Mkt: Café Mkt: Milho  \
Ticker          ^BVSP                CL=F      ZS=F       KC=F       ZC=F   
Date                                                                        
2003-01       10941.0           33.509998     564.0  65.300003     238.25   
2003-02       10281.0           36.599998     577.0  56.849998     231.75   
2003-03       11274.0           31.040001     574.5  58.650002     236.50   
2003-04       12557.0           25.799999     623.5  68.250000     232.75   
2003-05       13422.0           29.559999     624.5  58.349998     244.25   

Price   Mkt: Açúcar   Mkt: Ouro Mkt: S&P 500 Mkt: Mercados Emergentes  \
Ticker         SB=F        GC=F         ^SPX                      EEM   
Date                                                                    
2003-01        8.64  368.299988   855.700012                      NaN   
2003-02        8.90  350.200012   841.150024                      NaN   
2003-03        7.6

In [14]:
# Salva o all data em um arquivo CSV
all_data.to_csv("all_data.csv")